## 영화 추천을 위한 영화 데이터 및 리뷰정보 크롤링

1. 왓차피디아 
- https://pedia.watcha.com/ko-KR

2. 영화개봉정보 - 영화관입장권통합전산망
- https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do#none


In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests


from urllib.parse import quote

import time
#time.sleep을 위한 난수생성
import random

from tqdm import tqdm

# text 전처리
import re

# 마우스 커서 이동을 위한 패키지
import pyautogui

In [3]:
movie = pd.read_excel('data/영화정보 리스트_2024-03-19.xls', header=4)
movie.head(5)

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,남은 인생 10년,The Last 10 Years,2022.0,일본,장편,멜로/로맨스,개봉,후지이 미치히토,NaN
1,달짝지근해: 7510,HONEYSWEET,2022.0,한국,장편,코미디,개봉,이한,(주)무비락
2,폭로,Havana,2022.0,한국,장편,"범죄,스릴러",개봉,홍용호,(주)로그라인스튜디오
3,어디로 가고 싶으신가요,Where Would You Like to Go?,2023.0,한국,장편,드라마,개봉,김희정,(주)인벤트스톤
4,만해 한용운 님의침묵,Silence of Thy,2022.0,한국,장편,드라마,개봉,한명구,주식회사 시네마서울


In [4]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화명      1547 non-null   object 
 1   영화명(영문)  794 non-null    object 
 2   제작연도     1516 non-null   float64
 3   제작국가     1547 non-null   object 
 4   유형       1547 non-null   object 
 5   장르       1493 non-null   object 
 6   제작상태     1546 non-null   object 
 7   감독       1242 non-null   object 
 8   제작사      632 non-null    object 
dtypes: float64(1), object(8)
memory usage: 108.9+ KB


In [5]:
movie['장르'].fillna('', inplace=True)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_10516\3068823807.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movie['장르'].fillna('', inplace=True)


In [6]:
movie = movie[~movie['장르'].str.contains('성인')]

In [7]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 775 entries, 0 to 1546
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화명      775 non-null    object 
 1   영화명(영문)  658 non-null    object 
 2   제작연도     753 non-null    float64
 3   제작국가     775 non-null    object 
 4   유형       775 non-null    object 
 5   장르       775 non-null    object 
 6   제작상태     775 non-null    object 
 7   감독       624 non-null    object 
 8   제작사      257 non-null    object 
dtypes: float64(1), object(8)
memory usage: 60.5+ KB


In [8]:
movie['제작연도'].unique()

array([2022., 2023., 2020., 2021., 2004.,   nan, 2019., 1995., 1990.,
       1988., 1964., 1980., 2018., 2015., 1948., 1999., 1996.])

In [9]:
movie['제작연도'] =  movie['제작연도'].fillna(0).apply(lambda x: str(int(np.round(x))))

In [10]:
movie['장르'].unique()

array(['멜로/로맨스', '코미디', '범죄,스릴러', '드라마', '코미디,드라마,멜로/로맨스', '다큐멘터리',
       '사극,드라마', '멜로/로맨스,판타지,드라마', '판타지,드라마', '드라마,멜로/로맨스,범죄',
       '멜로/로맨스,코미디', '드라마,기타', '사극,액션', '드라마,멜로/로맨스,뮤지컬,공연', '애니메이션',
       '드라마,멜로/로맨스', '드라마,사극,스릴러,전쟁', '코미디,드라마', '공포(호러),코미디',
       '미스터리,드라마,스릴러', '드라마,미스터리', '드라마,액션,범죄', 'SF,드라마,스릴러',
       '드라마,판타지,멜로/로맨스', '공연,다큐멘터리', '', '미스터리,범죄,드라마', '스릴러',
       '멜로/로맨스,드라마', '공연', '공포(호러)', '멜로/로맨스,드라마,판타지', '드라마,범죄', '범죄',
       '판타지,어드벤처,애니메이션', '전쟁,액션', '스릴러,멜로/로맨스', '공포(호러),미스터리',
       '액션,어드벤처,판타지', '미스터리,스릴러,드라마', '액션', 'SF,액션', '범죄,액션',
       '공포(호러),스릴러', '코미디,액션', '미스터리,스릴러', '어드벤처,드라마,코미디',
       '애니메이션,어드벤처,판타지', '스릴러,드라마,코미디', '전쟁,드라마', '드라마,멜로/로맨스,가족',
       '액션,드라마,스릴러', '액션,드라마', '애니메이션,어드벤처', '범죄,드라마,스릴러',
       '애니메이션,판타지,어드벤처', '액션,스릴러,전쟁', '미스터리,공포(호러)', 'SF,스릴러',
       '다큐멘터리,드라마,가족', '판타지,액션', '다큐멘터리,범죄', '범죄,액션,스릴러', '스릴러,공포(호러)',
       'SF,액션,드라마', '서부극(웨스턴),코미디,액션', '액션,어드벤처,SF,스릴러', '액션,스릴러',
       '액션,범죄', '드라마,코미디', '

In [11]:
movie_list = movie['영화명'].to_list()
movie_list

['남은 인생 10년',
 '달짝지근해: 7510',
 '폭로',
 '어디로 가고 싶으신가요',
 '만해 한용운 님의침묵',
 '나비효과',
 '울지마 엄마',
 '연악: 나의 운명',
 '어느 날 그녀가 우주에서',
 '3일의 휴가',
 '흠뻑채운 젖은 속살',
 '탐욕에 젖은 섹스',
 '탐욕스런 동침',
 '타락한 유부녀의 유혹',
 '유부녀의 욕망 속살',
 '유부녀의 비밀스런 사정',
 '옆집 유부녀의 속사정',
 '아버님이 채운 유부녀',
 '자우림, 더 원더랜드',
 '위험사회',
 '서울의 봄',
 '싱글 인 서울',
 '콘크리트 유토피아',
 '노량: 죽음의 바다',
 '사랑의 불시착: 라이브 인 서울',
 '두다다쿵: 후후섬의 비밀',
 '나의 피투성이 연인',
 '듣보인간의 생존신고',
 '어른 김장하',
 '돼지의 최후',
 '겨울 이야기',
 '드림',
 '손',
 '비밀',
 '신세계로부터',
 '솔라 플라워',
 '르네에게',
 '익스트림 페스티벌',
 '다음 소희',
 '괴인',
 '돈 룩 디퍼',
 '사랑엔 정의가 없다',
 '그대들은 어떻게 살 것인가',
 '청춘 돼지는 외출하는 여동생의 꿈을 꾸지 않는다',
 '너와 나',
 '말하고 싶은 비밀',
 '상견니',
 '류이치 사카모토: 오퍼스',
 '절해고도',
 '미끼',
 '도티와 영원의 탑',
 '사위 사랑은 장모 4',
 '여배우들 : 바꿔할래?',
 '그녀의 취미생활',
 '라방',
 '오늘 출가합니다',
 '리향',
 '자매의 스와핑',
 '참을 수 없는 탐욕',
 '바지 내린 욕정녀',
 '몸으로 받은 상사',
 '우리 돈 벌레',
 '옆집 아저씨와 같은 침대에 누워 있다',
 '새엄마 잠깐 거긴 안돼',
 '멋대로 들어온 돌싱녀 : 하고 싶어요',
 '남편 말고 의붓아들 : 속궁합이 잘 맞아',
 '쉽게 넣는 그녀 무삭제판',
 '가랑이 채운 처형 무삭제판',
 '르네상스 필름 바이 비욘세',
 '아이엠 장모에요',
 '며느리의 반복 된 숨소리 : 나를 흥분하게 해

In [12]:
year_list = movie['제작연도'].to_list()
year_list

['2022',
 '2022',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2020',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2021',
 '2022',
 '2023',
 '2022',
 '2022',
 '2021',
 '2023',
 '2023',
 '2004',
 '2023',
 '2021',
 '2023',
 '2021',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2021',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2021',
 '2023',
 '2023',
 '2023',
 '2023',
 '0',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '0',
 '2023',
 '2023',
 '2022',
 '2022',
 '2023',
 '2023',
 '2022',
 '2022',
 '2020',
 '2022',
 '2021',
 '2022',
 '2022',
 '2020',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2019',
 '2023',
 '2023',
 '2022',
 '2023',
 '2022',
 '2023',
 '2022',
 '2021',
 '2023',
 '2022',
 '2023'

In [79]:
### 기본틀 완성

base_url = 'https://pedia.watcha.com/ko-KR/searches/movies?query='
movie_title = movie['영화명'][21]
movie_year = movie['제작연도'][21]
url = base_url + movie_title


driver = webdriver.Chrome()

driver.get(url)
driver.set_window_position(-1800,10)

# 화면이 뜰때까지 기다리기
driver.implicitly_wait(10)
# 창 최대화
# driver.maximize_window()
# time.sleep(2.4)


### 같은 제목의 영화에서 제작연도가 같은 영화를 선택
movie_lis = driver.find_element(By.CLASS_NAME, 'e1493pgd7').find_elements(By.CLASS_NAME, 'e1ic68ft1')

cnt = 0
for li in movie_lis:
    li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0]
    if li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0] != movie_year:
        cnt += 1
    else :
        print(li.find_element(By.CLASS_NAME, 'e1gexyxq4').text)
        print(cnt)
        break

#print(lis[cnt].find_element(By.CLASS_NAME, 'e1gexyxq4').text)

# 제작연도가 같은 영화가 없을 경우에 첫번째 영화를 선택
try:
    movie_lis[cnt].click()
except:
    movie_lis[0].click()

# 영화 소개
print(driver.find_element(By.CLASS_NAME, 'e1yew2864').text.strip())

# 감독/출연진
person_lis = driver.find_element(By.CLASS_NAME, 'e1n40jsk1').find_elements(By.CLASS_NAME, 'e4f2vpe2')
for person in person_lis:
    print(person.find_element(By.TAG_NAME, 'a').get_attribute('title'))

# 전체 평점
print(driver.find_element(By.CLASS_NAME, 'e15eo6m110').text)

# 전체 평점 리뷰어수
print(driver.find_element(By.CLASS_NAME, 'e15eo6m19').text.split('\n')[-1])
time.sleep(round(random.uniform(1.3, 2.2), 3))

## 리뷰 페이지 이동
driver.find_element(By.CLASS_NAME, 'ehapiph0').click()
time.sleep(round(random.uniform(1.3, 2.2), 3))

# 스크롤 다운
for _ in range(5):
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(1.0, 1.5), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(0.3, 1.2), 3))
time.sleep(round(random.uniform(1.3, 2.2), 3))


review_list = driver.find_elements(By.CLASS_NAME, 'egj9y8a4')
for review in review_list:
    # 사용자 명
    print(review.find_element(By.CLASS_NAME, 'e10cf2lr1').text)

    # 평점
    print(review.find_element(By.CLASS_NAME, 'egj9y8a0').text)

    # 리뷰
    print(review.find_element(By.CLASS_NAME, 'e1hvy88212').text.strip().replace('\n', ''))
time.sleep(round(random.uniform(1.3, 2.2), 3))

1979년 12월 12일, 수도 서울 군사반란 발생
그날, 대한민국의 운명이 바뀌었다.

대한민국을 뒤흔든 10월 26일 이후, 서울에 새로운 바람이 불어온 것도 잠시 12월 12일, 보안사령관 전두광이 반란을 일으키고 군 내 사조직을 총동원하여 최전선의 전방부대까지 서울로 불러들인다. 권력에 눈이 먼 전두광의 반란군과 이에 맞선 수도경비사령관 이태신을 비롯한 진압군 사이, 일촉즉발의 9시간이 흘러가는데…
  
목숨을 건 두 세력의 팽팽한 대립
오늘 밤, 대한민국 수도에서 가장 치열한 전쟁이 펼쳐진다!
김성수(감독)
황정민(주연 | 전두광)
정우성(주연 | 이태신)
이성민(조연 | 참모총장 정상호)
박해준(조연 | 9사단장 노태건)
김성균(조연 | 헌병감 김준엽)
김의성(조연 | 국방장관)
정동환(조연 | 대통령 최한규)
안내상(조연 | 1군단장 한영구)
유성주(조연 | 참모차장 민성배)
최병모(조연 | 2공수 여단장 도희철 )
박훈(조연 | 비서실장 문일평)
이재윤(조연 | 수사과장 임학주)
김성오(조연 | 4공수 여단장 김창세)
남윤호(조연 | 수경사 작전참모 강동찬)
홍서준(조연 | 보안사 인사참모 하창수)
안세호(조연 | 30경비단장 장민기)
정형석(조연 | 8공수 여단장 박기홍)
박정학(조연 | 30사단장 모상돈)
박원상(조연 | 3군사사령관 고재영)
박민이(조연 | 33경비단장 진영도)
염동헌(조연 | 군수차관보 배송학)
전진기(조연 | 수도군단장 현치성)
최원경(조연 | 헌병단장 원경)
3.9
(9.2만명)
WE
5.0

이동진 평론가
3.5
야만과 무능의 그 겨울밤에 대한 분노가 시종 팽팽한 긴장감 속에서 펄펄 끓는다.
박준영 (영화 담다)
4.5
일제강점기때 독립을 끝까지 외친 이유는 우리가 저항했다는 것을 기록으로 남기기 위해서였고 12.12 사태도 마찬가지일 것이다. '승자가 역사를 규정한다'는 말이 있지만 이제는 그런 말이 통하지 않게 해야하지 않을까 싶다.허구의 이야기로 각색되었지만 실존 인물과 실제 한국 현대사이기에 우리는 신군부 세력

In [25]:
### 데이터 프레임화

base_url = 'https://pedia.watcha.com/ko-KR/searches/movies?query='

movie_info = []
movie_review = []

#for i in range(len(movie_list)):
for i in range(10):
    movie_title = movie['영화명'][i]
    movie_year = movie['제작연도'][i]

    url = base_url + movie_title

    driver = webdriver.Chrome()

    driver.get(url)
    driver.set_window_position(-1800,10)

    # 화면이 뜰때까지 기다리기
    driver.implicitly_wait(10)
    # 창 최대화
    # driver.maximize_window()
    # time.sleep(2.4)
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    #################################################################################################################
    ### 같은 제목의 영화에서 제작연도가 같은 영화를 선택
    same_lis = driver.find_element(By.CLASS_NAME, 'e1493pgd7').find_elements(By.CLASS_NAME, 'e1ic68ft1')

    cnt = 0
    for li in same_lis:
        li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0]
        if li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0] != movie_year:
            cnt += 1
        else :
            print(li.find_element(By.CLASS_NAME, 'e1gexyxq4').text.strip())
            print(cnt)
            break

    # 제작연도가 같은 영화가 없을 경우에 첫번째 영화를 선택
    try:
        same_lis[cnt].click()
    except:
        same_lis[0].click()

    #################################################################################################################
    ### 영화 정보 크롤링
    # 영화 소개
    print(driver.find_element(By.CLASS_NAME, 'e1yew2864').text.strip())

    # 감독/출연진
    person_lis = driver.find_element(By.CLASS_NAME, 'e1n40jsk1').find_elements(By.CLASS_NAME, 'e4f2vpe2')
    for person in person_lis:
        print(person.find_element(By.TAG_NAME, 'a').get_attribute('title'))

    # 전체 평점
    print(driver.find_element(By.CLASS_NAME, 'e15eo6m110').text)

    # 전체 평점 리뷰어수
    print(driver.find_element(By.CLASS_NAME, 'e15eo6m19').text.split('\n')[-1])
    

    # 정보 1
    print(driver.find_element(By.CLASS_NAME, 'e1yew28613').find_elements(By.CLASS_NAME, 'e1yew28611')[1].text)

    # 정보 2
    print(driver.find_element(By.CLASS_NAME, 'e1yew28613').find_elements(By.CLASS_NAME, 'e1yew28611')[2].text)

    time.sleep(round(random.uniform(1.3, 2.2), 3))
    
    #################################################################################################################
    # 리뷰 페이지 이동
    driver.find_element(By.CLASS_NAME, 'ehapiph0').click()
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    # 스크롤 다운
    for _ in range(5):
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
        driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
        time.sleep(round(random.uniform(1.0, 1.5), 3))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
        time.sleep(round(random.uniform(0.3, 1.2), 3))
    time.sleep(round(random.uniform(1.3, 2.2), 3))


    review_list = driver.find_elements(By.CLASS_NAME, 'egj9y8a4')
    for review in review_list:
        
        # 평점이 없는 경우 발생 - ex)보고싶어요
        try:
            # 평점
            print(review.find_element(By.CLASS_NAME, 'egj9y8a0').text.strip())

            #사용자명
            print(review.find_element(By.CLASS_NAME, 'e10cf2lr1').text.strip())

            # 리뷰
            print(review.find_element(By.CLASS_NAME, 'e1hvy88212').text.strip().replace('\n', ''))
        except:
            continue
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    driver.close()



남은 인생 10년
0
매일 밤, 눈 감기가 두려웠던 난치병 환자 ‘마츠리’
매일 아침, 눈 뜨기가 두려웠던 외톨이 ‘카즈토’ 
운명은 정반대에 있던 두 사람을 마주 보게 한다.

처음 만난 봄,
많이 웃은 여름,
더욱 깊어진 가을,
소중해서 아팠던 겨울까지
남은 인생을 진정으로 사랑하게 해준 너에게 감사해
후지 미치히토(감독)
고마츠 나나(주연 | 마츠리)
사카구치 켄타로(주연 | 카즈토)
야마다 유키(조연)
나오(조연)
이구치 사토루(조연)
쿠로키 하루(조연)
타나카 테츠지(조연)
하라 히데코(조연)
릴리 프랭키(조연)
마츠시게 유타카(조연)
3.3
(5,849명)
2022 · 드라마/로맨스 · 일본
2시간 4분 · 12세
3.0
재원
영화를 보며 눈시울이 붉어졌던 건, 내게도 한땐내가 좀 더 괜찮은 인간이 되고 싶게 해준 사람이있었기 때문이겠지. 그 사람과의 작별이 얼마나쓰라렸는지 절실히 기억하기 때문이겠지.
3.5
박준영 (영화 담다)
영롱하고 유려한 피아노와 현악 음악이 흐르고 자연광과 아름다운 풍광이 서두부터 마음을 잔잔하게 만드는 일본의 서정성이 가득한 로맨스 멜로 영화 <남은 인생 10년>.누군가 실패를 했을 때 다음이 중요하다는 조언을 해주곤 한다. 하지만 다음 같은 건 없다. 후회를 하든 안하든 인생은 단 한 번뿐이다. 지금에 최선을 다하며 살아가야 한다. 앞으로 10년밖에 살지 못한다면 당신은 무엇을 할 건가요? 오늘과 내일에 대해 생각해보게 된다.
3.5
오늘은너에게로
정해진 운명은 야속했지만, 둘의 추억만큼은 행복했기를.
2.5
뭅먼트
누군가를 살아내게 만드는 것은, 눈앞에서 점점 흐려지는 커다란 희망이 아니라 눈앞에서 돌연히 흩날리는 작은 소망.
2.5
JY
부슬부슬 봄비처럼 흩날리면 뭐하나결국 2시간동안 내리면흥건해지는건 매한가지인데
3.0
진태
인물이 남긴 잔상이 영화를 기억하게 한다
3.0
INWA
연기, 영상, 음악, 감정 각각의 요소는모두 기대 이상인데전체로 봤을 때, 결말까지 보았을 때어째선지 그 이상의 시너지를 내진 못하는 

In [30]:
### 데이터 프레임화

base_url = 'https://pedia.watcha.com/ko-KR/searches/movies?query='

movie_info = []
movie_review = []

#for i in range(len(movie_list)):
for i in range(100, 120):
    movie_title = movie['영화명'][i]
    movie_year = movie['제작연도'][i]

    url = base_url + movie_title

    driver = webdriver.Chrome()

    driver.get(url)
    driver.set_window_position(-1800,10)

    # 화면이 뜰때까지 기다리기
    driver.implicitly_wait(10)
    # 창 최대화
    # driver.maximize_window()
    # time.sleep(2.4)
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    #################################################################################################################
    ### 같은 제목의 영화에서 제작연도가 같은 영화를 선택
    same_lis = driver.find_element(By.CLASS_NAME, 'e1493pgd7').find_elements(By.CLASS_NAME, 'e1ic68ft1')

    cnt = 0
    for li in same_lis:
        li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0]
        if li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0] != movie_year:
            cnt += 1
        else :
            print(li.find_element(By.CLASS_NAME, 'e1gexyxq4').text.strip())
            print(cnt)
            break

    # 제작연도가 같은 영화가 없을 경우에 첫번째 영화를 선택
    try:
        same_lis[cnt].click()
    except:
        same_lis[0].click()

    #################################################################################################################
    ### 영화 정보 크롤링
    # 영화 소개
    summary = driver.find_element(By.CLASS_NAME, 'e1yew2864').text.strip()

    # 감독/출연진
    actor_list = []
    person_lis = driver.find_element(By.CLASS_NAME, 'e1n40jsk1').find_elements(By.CLASS_NAME, 'e4f2vpe2')
    for person in person_lis:
        actor_list.append(person.find_element(By.TAG_NAME, 'a').get_attribute('title'))

    # 전체 평점
    total_rating = driver.find_element(By.CLASS_NAME, 'e15eo6m110').text

    # 전체 평점 리뷰어수
    rating_cnt = driver.find_element(By.CLASS_NAME, 'e15eo6m19').text.split('\n')[-1]
    

    # 정보 1
    info1 = driver.find_element(By.CLASS_NAME, 'e1yew28613').find_elements(By.CLASS_NAME, 'e1yew28611')[1].text

    # 정보 2
    info2 = driver.find_element(By.CLASS_NAME, 'e1yew28613').find_elements(By.CLASS_NAME, 'e1yew28611')[2].text

    time.sleep(round(random.uniform(1.3, 2.2), 3))
    movie_info.append({'title' : movie_title,
                       'movie_num' : i,
                       'summary' : summary,
                       'actor_list': actor_list,
                       'rating' : total_rating,
                       'rating_cnt' : rating_cnt,
                       'info1' : info1,
                       'info2' : info2})
    
    #################################################################################################################
    # 리뷰 페이지 이동
    driver.find_element(By.CLASS_NAME, 'ehapiph0').click()
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    # 스크롤 다운
    for _ in range(5):
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
        driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
        time.sleep(round(random.uniform(1.0, 1.5), 3))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
        time.sleep(round(random.uniform(0.3, 1.2), 3))
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    num = 0
    review_list = driver.find_elements(By.CLASS_NAME, 'egj9y8a4')
    for review_li in review_list:
        
        # 평점이 없는 경우 발생 - ex)보고싶어요
        try:
            # 평점
            review_rating = review_li.find_element(By.CLASS_NAME, 'egj9y8a0').text.strip()

            #사용자명
            reviewer = review_li.find_element(By.CLASS_NAME, 'e10cf2lr1').text.strip()

            # 리뷰
            review = review_li.find_element(By.CLASS_NAME, 'e1hvy88212').text.strip().replace('\n', '')

            # num
            num += 1

            movie_review.append({'title' : movie_title,
                                 'movie_num' : i,
                                 'review_num' : num,
                                 'reviewer' : reviewer,
                                 'review_rating' : review_rating,
                                 'review' : review})
        except:
            continue
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    driver.close()



KeyError: 100

0           남은 인생 10년
1         달짝지근해: 7510
2                  폭로
3        어디로 가고 싶으신가요
4         만해 한용운 님의침묵
            ...      
1542             패닉 런
1543    처음 꽃향기를 만난 순간
1544     뮤지컬 번지점프를 하다
1545          뮤지컬 배니싱
1546           트루 마더스
Name: 영화명, Length: 775, dtype: object

In [28]:
pd.DataFrame(movie_info)

,title,summary,actor_list,rating,rating_cnt,info1,info2
0,남은 인생 10년,"매일 밤, 눈 감기가 두려웠던 난치병 환자 ‘마츠리’\n매일 아침, 눈 뜨기가 두려...","[후지 미치히토(감독), 고마츠 나나(주연 | 마츠리), 사카구치 켄타로(주연 | ...",3.3,"(5,849명)",2022 · 드라마/로맨스 · 일본,2시간 4분 · 12세
1,달짝지근해: 7510,"타고난 미각 100%, 현실 감각은 0% 제과 연구원 ‘치호’(유해진). 과자밖에 ...","[이한(감독), 유해진(주연 | 치호), 김희선(주연 | 일영), 차인표(조연 | ...",2.9,(1.2만명),2022 · 코미디/드라마 · 한국,1시간 58분 · 12세
2,폭로,신출내기 변호인과 은밀한 피고인의 진실게임 \n모든 것을 뒤집는 ‘폭로’! \n\n...,"[홍용호(감독), 유다인(주연 | 윤아), 강민혁(주연 | 정민), 공상아(조연 |...",2.1,(673명),2022 · 드라마 · 한국,1시간 41분 · 12세
3,어디로 가고 싶으신가요,"어느 봄날, 갑작스러운 사고로 남편인 ‘도경’을 잃은 ‘명지’는 슬픔에서 벗어나기 ...","[김희정(감독), 정민주(주연 | 지은), 박하선(주연), 김남희(조연), 전석호(...",2.8,(851명),"2023 · 드라마 · 한국, 폴란드",1시간 43분 · 12세
4,만해 한용운 님의침묵,1.만해 한용운 (1899년~1944년) 독립운동가인가 하면 시인이요 시인가면 대사...,"[한명구(감독), 오안진(주연 | 만해 한용운), 이소망(조연 | 만해 책사 화영)...",2.0,(12명),2022 · 드라마 · 한국,2시간 24분 · 15세
5,나비효과,"노래면 노래, 춤이면 춤! 못 하는 게 없는 밴드 ‘나비효과’의 보컬 ‘주연’(홍수...","[민준기(감독), 전원창(감독), 홍수아(주연), 윤성모(주연), 안정훈(주연), ...",1.9,(43명),2022 · 드라마 · 한국,1시간 34분 · 12세
6,울지마 엄마,"""하나님. 조금만 더 살게 해주세요..."" \n \n행복한 나날을 보내던 세 가정에...",[이호경(감독)],2.9,(42명),2022 · 다큐멘터리 · 한국,1시간 18분 · 전체
7,연악: 나의 운명,"조선 초기, 예악으로 나라의 기틀을 세우던 시대. 대왕 세종은 천재 음악가 ‘박연’...","[박한열(감독), 정욱(주연 | 박연), 송민경(주연 | 백아), 손건우(주연), ...",1.8,(24명),2022 · 시대극/드라마 · 한국,1시간 53분 · 전체
8,어느 날 그녀가 우주에서,만년 취준생 ♥ 외계 소녀 \n은하계를 넘어선 우주적 교감이 시작된다! \n \n머...,"[구상범(감독), 백서빈(주연 | 석민), 신연서(주연 | 나은), 서지영(조연 |...",3.0,(777명),2022 · 로맨틱 코미디/로맨스 · 한국,1시간 43분 · 12세
9,3일의 휴가,“따님은 어머님을 보거나 목소리를 들을 수 없고요.\n그냥 따님의 행복한 기억만 담...,"[육상효(감독), 김해숙(주연 | 박복자), 신민아(주연 | 방진주), 강기영(조연...",2.9,"(4,780명)",2023 · 판타지/드라마/가족 · 한국,1시간 45분 · 12세


In [29]:
pd.DataFrame(movie_review)

,title,review_num,reviewer,review
0,남은 인생 10년,1,재원,"영화를 보며 눈시울이 붉어졌던 건, 내게도 한땐내가 좀 더 괜찮은 인간이 되고 싶게..."
1,남은 인생 10년,2,박준영 (영화 담다),영롱하고 유려한 피아노와 현악 음악이 흐르고 자연광과 아름다운 풍광이 서두부터 마음...
2,남은 인생 10년,3,오늘은너에게로,"정해진 운명은 야속했지만, 둘의 추억만큼은 행복했기를."
3,남은 인생 10년,4,뭅먼트,"누군가를 살아내게 만드는 것은, 눈앞에서 점점 흐려지는 커다란 희망이 아니라 눈앞에..."
4,남은 인생 10년,5,JY,부슬부슬 봄비처럼 흩날리면 뭐하나결국 2시간동안 내리면흥건해지는건 매한가지인데
...,...,...,...,...
505,3일의 휴가,93,Jamongsoda,"졸았다.. 스팸김치찌개만 생각남..노스페이스, 커피, 소주, 맥주, 스팸, 맛소금…..."
506,3일의 휴가,94,심우영,치유의 대상은 판타지가 아니라 현실이 되어야 한다. 현실을 치유하지 못하는 판타지는...
507,3일의 휴가,95,밀크쉐이크,30분짜리 단편이로 뽑을 소재를 1시간 40분으로 늘려놓은 느낌
508,3일의 휴가,96,박준형,"뻔한 이야기에, 더 뻔한 연출을김해숙, 신민아 두 배우가 살려낸다.엄마와의 <리틀 ..."
